In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.columns.values

#df<- The dataset has 18259 rows, 9 columns

array(['average_rate_per_night', 'bedrooms_count', 'city',
       'date_of_listing', 'description', 'latitude', 'longitude', 'title',
       'url'], dtype=object)

In [3]:
df[:3]

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...


In [4]:
mask_Austin = (df['city']=='Austin')
df= df[mask_Austin].dropna(axis=0,how = 'any')

df.shape #<- The dataset has 1477 listings for the city of Austin

(1477, 9)

In [5]:
#Get integer version of average rate
def average_rate(s):
    s=s.strip('$')
    return int(s)
df['average_rate']=df['average_rate_per_night'].map(average_rate)

In [6]:
#Get integer version of bedrooms

def bedroom_count(s):
    if s=='Studio':
        return 0
    else:
        return int(s)

df['num_of_bedrooms']= df['bedrooms_count'].map(bedroom_count)

In [7]:
df2 = df.drop('average_rate_per_night',axis=1).drop('bedrooms_count',axis=1)
cols2 = df2.columns.tolist()
cols2 = cols2[-2:]+cols2[:-2]
df2=df2[cols2]
df2.rename(columns={'average_rate':'average_rate($)'},inplace=True)


In [8]:
df2['Listing Month']= df2['date_of_listing'].astype(str).str.rsplit(' ',1).str[0]
df2['Listing Year'] = df2['date_of_listing'].astype(str).str.rsplit(' ',1).str[-1]

df2[:3]

,average_rate($),num_of_bedrooms,city,date_of_listing,description,latitude,longitude,title,url,Listing Month,Listing Year
16,82,2,Austin,December 2015,"Cute two bedroom with lots of windows, sunny b...",30.309522,-97.731710,Cosy 2 Bedroom in Hyde Park!,https://www.airbnb.com/rooms/18550276?location...,December,2015
28,110,1,Austin,September 2015,Comfortable Traditional Interior Furniture in ...,30.244296,-97.774469,Fabulous Central Austin Condo!,https://www.airbnb.com/rooms/13571814?location...,September,2015
33,119,1,Austin,August 2010,"Just minutes from Austin's main attractions, e...",30.300382,-97.778858,Sunny Central Austin Apartment,https://www.airbnb.com/rooms/46929?location=Co...,August,2010


In [9]:
df2.dtypes


average_rate($)      int64
num_of_bedrooms      int64
city                object
date_of_listing     object
description         object
latitude           float64
longitude          float64
title               object
url                 object
Listing Month       object
Listing Year        object
dtype: object

In [10]:
#Grab full address of properties  
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key='PRIVATE')

def get_addy(s):
    locations = geolocator.reverse(s)
    return locations[0].address
    

df2['latitude']=df2['latitude'].astype(str)
df2['longitude']=df2['longitude'].astype(str)       
df2['lat_long']= df2['latitude'].astype(str) + ',' + df2['longitude'].astype(str)
df2['FullAddy']=df2['lat_long'].map(get_addy)


/Users/GrantZhong/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: GoogleV3.reverse: default value for `exactly_one` argument will become True in geopy 2.0. Specify `exactly_one=False` as the argument explicitly to get rid of this warning.
  


GeocoderTimedOut: Service timed out

In [ ]:
df_austin=df2.copy()
df_austin

In [ ]:
freq_of_rate = df2['average_rate($)'].value_counts().sort_values(ascending=False)
freq_of_rate[:20].plot(kind='bar')
xlabel('Average Rate Per Night ($)')
ylabel('Freq Of Rate')

In [ ]:
freq_listing_month = df2['Listing Month'].value_counts().sort_values(ascending=False)
freq_listing_month.plot(kind='bar')
xlabel('Listing Month')
ylabel('Freq Of Listing')

In [ ]:
freq_listing_year = df2['Listing Year'].value_counts().sort_values(ascending=False)
freq_listing_year.plot(kind='bar')
xlabel('Listing Year')
ylabel('Freq Of Listing')